In [1]:
# import reddit api wrapper
import praw
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import numpy as np

import datetime
import tldextract

# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at the top level folder
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

/Users/prcork/Georgetown/Fall2022/DS3/final_project


In [2]:
%autoreload
from functions import submissions

Version 7.6.0 of praw is outdated. Version 7.6.1 was released Friday November 11, 2022.


In [3]:
# display number of subscribers for both subreddits
print(submissions.get_subscribers('democrats'))
print(submissions.get_subscribers('conservative'))

432051
1019549


In [4]:
# get 1000 highest upvoted submissions from the last month
dems = submissions.get_submissions(subreddit="democrats", limit=1000)
con = submissions.get_submissions(subreddit="conservative", limit=1000)

In [5]:
# print stats to include in summary tables

print("r/democrats")
print(dems.shape)
print(dems.ups.mean())
print(dems.num_comments.mean())
print()
print("r/conservative")
print(con.shape)
print(con.ups.mean())
print(con.num_comments.mean())


r/democrats
(782, 8)
290.18158567774935
31.812020460358056

r/conservative
(1000, 8)
718.761
221.111


In [6]:
# categorize and merge subreddits for preprocessing

dems['subreddit'] = 'democrats'
con['subreddit'] = 'conservative'

df = pd.concat([dems, con])

In [7]:
# convert UTC to datetime, then store only date
df['date'] = df['created_utc'].apply(lambda utc_entry: datetime.datetime.utcfromtimestamp(utc_entry))
df['date'] = df['date'].dt.date
df = df.query("date >= datetime.date(year=2022, month=8, day=1)")

# if the last four characters of the url are an image filetype, categorize the submission as an image
df['image'] = np.where(df.url.str[-4:].isin(['.jpg', '.png', '.gif']), 1, 0)

# split the url into sub, domain, and extension, store only domain
df['domain'] = df['url'].apply(lambda x: tldextract.extract(x).domain)

print(df.shape)
df.sample(5)

(1782, 12)


,submission_id,title,created_utc,is_self,url,ups,upvote_ratio,num_comments,subreddit,date,image,domain
0,yulby9,Louis DeJoy Can Be Fired As Biden Now Has A Ma...,1668390036.00,False,https://www.politicususa.com/2022/05/13/louis-...,19564,0.95,429,democrats,2022-11-14,0,politicususa
4,ysdxro,Another Friday Feel Good Moment...,1668181172.00,False,https://i.redd.it/yk9gw0l3vdz91.jpg,2794,0.97,103,democrats,2022-11-11,1,redd
318,z2oqij,Herschel Walker's 'erection' remark sparks tor...,1669209001.00,False,https://www.newsweek.com/herschel-walkers-erec...,155,0.98,24,democrats,2022-11-23,0,newsweek
341,z3hunt,Pennsylvania school district votes to defy law...,1669290204.00,False,https://www.foxnews.com/media/pennsylvania-sch...,596,0.86,80,conservative,2022-11-24,0,foxnews
582,z5jfmo,Georgia Supreme Court allows early voting on p...,1669500520.00,False,https://www.cnn.com/2022/11/23/politics/georgi...,36,1.00,1,democrats,2022-11-26,0,cnn


In [8]:
df.drop(['created_utc', 'url'], axis=1, inplace=True)
df.to_csv("data/reddit_output.csv", index=False)